# 环境准备

In [ ]:
!pip install ../input/dlib-whl/dlib-19.19.0-cp36-cp36m-linux_x86_64.whl

In [1]:
import cv2
import numpy as np
import os
import dlib
import glob
import pandas as pd
from matplotlib import pyplot as plt
import pickle
from tqdm import tqdm
from pathlib import Path

# 变量声明

In [ ]:
test_video_path = "/kaggle/input/deepfake-detection-challenge/test_videos/"

# 工具函数

In [ ]:
def extract_faces(path_to_video,  save_dir=".", count=5):
    detector = dlib.get_frontal_face_detector()
    camera = cv2.VideoCapture(path_to_video)
    
    path = Path(path_to_video)
#     if not camera.isOpened():
#         print("cannot open camera")
#         exit(0)
    faces_count = 0
    faces_path = []
    while faces_count < count:
        try:
            ret, frame = camera.read()
            frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w = frame_new.shape[0], frame_new.shape[1]
            # print(w, h)
            # 检测脸部
            dets = detector(frame_new, 1)
            if len(dets) > 0:
                # 查找脸部位置
                print("Number of faces detected: {}".format(len(dets)))
                for i, face in enumerate(dets):
                    #保存脸部图片,注意这里+ - 是根据dlib对脸部的定位来的，我这里选的50很适合我这个视频
                    img1=frame[max(face.top()-50, 0):min(face.bottom()+50, h),max(face.left()-50, 0):min(face.right()+50, w)]
                    crop_size = (224,224)
                    print(f"{max(face.top()-50, 0)}:{min(face.bottom()+50, h-1)},{max(face.left()-50, 0)}:{min(face.right()+50, w-1)}")
                    img1 = cv2.resize(img1, crop_size, interpolation = cv2.INTER_CUBIC)
                    dst_path = f"{save_dir}/{path.stem}_{faces_count:02}.jpg"
                    cv2.imwrite(dst_path, img1)
                    faces_count += 1
                    faces_path.append(dst_path)
        except:
            break
    cv2.destroyAllWindows()
    return faces_path


In [ ]:
extract_faces("/kaggle/input/deepfake-detection-challenge/test_videos/ahjnxtiamx.mp4")

In [ ]:
# from https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/

def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (x.max()-x.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]
    radial_prof = tbin / nr

    return radial_prof

In [ ]:
def get_psd1D(path_to_img, epsilon = 1e-8):
    img = cv2.imread(path_to_img,0)
    # Calculate FFT
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)
    return psd1D

# 预测

In [ ]:
#包导入
%matplotlib inline
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

In [ ]:
def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [ ]:
model = load_object("../input/svm-model/svm.pkl")

In [ ]:
psd1D = get_psd1D("ahjnxtiamx_00.jpg")
model.predict_proba(psd1D[:156].reshape(-1, 156))

In [ ]:
df = pd.DataFrame(columns=['filename', 'label'])
for i, video in enumerate(os.listdir(test_video_path)):
    print(i, video)
    imgs = extract_faces(f"{test_video_path}/{video}", count=1)
    print(imgs)
    if len(imgs)<1:
        proba = 1
    else:
        probas = []
        for img in imgs:
            psd1D_list = get_psd1D(img)
            psd1D = psd1D[:156].reshape(-1, 156)
#             probas.append(model.predict(psd1D)[0])
            probas.append(model.predict_proba(psd1D)[0][0])
        proba = np.mean(probas)
    print(proba)
    df.loc[i] = [video, proba]
df

In [ ]:
df.to_csv("submission.csv", index=False)